In [55]:
%pip install nltk
%pip install --upgrade transformers accelerate 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%` not found.


In [1]:
import torch
from torch.utils.data import  DataLoader,TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BlenderbotForCausalLM,AutoTokenizer,AdamW,Trainer,TrainingArguments,DataCollatorForLanguageModeling
from datasets import Dataset,DatasetDict
from peft import LoraConfig,TaskType,get_peft_model
# from google.colab import drive
import bitsandbytes as bnb
from sklearn.metrics import precision_score, recall_score
from nltk.translate.bleu_score import corpus_bleu
# import wandb

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of 

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [8]:
with open('../../../dataStuff/bookStuff/osBookDoc.txt','r',encoding='utf-8', errors='ignore') as p:
    s=p.readlines()

In [9]:
s=s[706:]

In [10]:
len(s)

20409

In [11]:
for i in range(len(s)):
    try:
        s.remove('\n')
    except:
        continue

In [28]:

for i in s:
    if(len(i)<190):
        s.remove(i)
    else :
        continue


In [29]:
len(s)

3861

In [30]:
dataDict={'context':s}

In [31]:
data=pd.DataFrame(dataDict)

In [32]:
data

,context
0,"Internally, operating systems vary greatly ..."
1,Because an operating system is large and co...
2,An operating system is a program that manages ...
3,Before we can explore the details of compu...
4,Because an operating system is large and co...
...,...
3856,[Rashid and Robertson (1981)] R. Rashid and G....
3857,"[Tevanian et al. (1987b)] A. Tevanian, Jr., R...."
3858,"Figure 1.11: From Hennesy and Patterson, Comp..."
3859,Figure 12.9: From Leffler/McKusick/Karels/Qua...


In [33]:
train,val=train_test_split(data,random_state=42)

In [34]:
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [36]:
model=BlenderbotForCausalLM.from_pretrained('facebook/blenderbot-400M-distill').to(device)
tokenizer=AutoTokenizer.from_pretrained('facebook/blenderbot-400M-distill')
datacollector=DataCollatorForLanguageModeling(tokenizer,mlm=False)

,context
370,systems do not have conflicting license agreem...
1091,"In Figure 6.8, we illustrate a Pthread sche..."
1874,"In the SCAN algorithm, the disk arm starts at ..."
932,"cores. However, multithreaded applications pre..."
729,It is easy to confuse TLS with local variab...
...,...
1130,"6.17 is 75 percent; therefore, the rate-monoto..."
1294,"<P1, P2, ..., Pn> is a safe sequence for the c..."
860,A solution to either problem may result in ...
3507,"As a final compatibility measure, several e..."


In [38]:
train=Dataset.from_pandas(train)
val=Dataset.from_pandas(val)

In [44]:
train['context']

['systems do not have conflicting license agreements. For example, DTrace has been added to Mac OS X and FreeBSD and will likely spread further due to its unique capabilities. Other operating systems, especially the Linux derivatives, are adding kernel-tracing functionality as well. Still other operating systems are beginning to include performance and tracing tools fostered by research at various institutions, including the Paradyn project.\n',
 '   In Figure 6.8, we illustrate a Pthread scheduling API.   The   pro- gram first determines the existing contention scope and sets it to PTHREAD SCOPE SYSTEM. It then creates five separate threads that will run using the SCS scheduling policy. Note that on some systems, only certain contention scope values are allowed. For example, Linux and Mac OS X systems allow only PTHREAD SCOPE SYSTEM.\n',
 'In the SCAN algorithm, the disk arm starts at one end of the disk and moves toward the other end, servicing requests as it reaches each cylinder, u

In [41]:
data=DatasetDict({
    "train":train,
    "validation":val
})

In [42]:
data

DatasetDict({
    train: Dataset({
        features: ['context', '__index_level_0__'],
        num_rows: 2895
    })
    validation: Dataset({
        features: ['context', '__index_level_0__'],
        num_rows: 966
    })
})

In [46]:
def tokenize_function(data):
    return tokenizer(data['context'],max_length=256)

In [47]:
token_data=data.map(tokenize_function,batched=True,batch_size=32,remove_columns=data['train'].column_names)

Map: 100%|██████████| 966/966 [00:00<00:00, 1738.67 examples/s]


In [48]:
token_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2895
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 966
    })
})

In [53]:
args=TrainingArguments("Casual_LLM OS"
                       ,evaluation_strategy="epoch",
                       save_strategy="epoch",
                       learning_rate=2e-5,
                       num_train_epochs=50,
                       weight_decay=0.01,
                       fp16=True,
                       per_device_train_batch_size=4,
                       per_device_eval_batch_size=4)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [54]:
trainer=Trainer(model=model,
                tokenizer=tokenizer,
                args=args,
                data_collator=datacollector,
                train_dataset=token_data['train'],
                eval_dataset=token_data['validation'])

ValueError: Tried to use `fp16` but it is not supported on cpu

In [ ]:
import  math
init_res=trainer.evaluate()
print(init_res)


In [ ]:
test_prompt = "What is operating system?"
test_prompt2 = "what are the scheduling algorithms??"
test_prompt3 = "What is the best way to find the shortest seeking time?"


In [ ]:
#Use the model in a pipeline to generate text.
from transformers import pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)


result = text_generator(test_prompt, max_length=100, num_return_sequences=1,temperature=1)
print(f"Baseline {'facebook/blenderbot-400M-distill'} generated result: {test_prompt}...{result[0]['generated_text']}")


In [ ]:
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results['eval_loss'])
eval_results['perplexity'] = perplexity

print(f"Fine-tuned {'Casual_LLM OS'} Results: Perplexity: {perplexity:.2f}")

In [ ]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
result = text_generator(test_prompt, max_length=100, num_return_sequences=1,temperature=0.7)
print(f"Fine-tuned {'Casual_LLM OS'} generated result: {test_prompt}...{result[0]['generated_text']}")
